In [22]:
import pandas as pd
import os
import warnings
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import ast

warnings.filterwarnings("ignore")
plt.style.use('dark_background')

In [23]:
# Use this chunk to generate the dataset if not already created
# df = pd.DataFrame()
# files = os.listdir("./data/patients_new")
# for file in files:
    # patient = pd.read_json(os.path.join("./data/patients_new/", file))
    # df = pd.concat([df, patient], ignore_index=True)
# df.reset_index(drop=True)
# df.to_csv("./collated_dataset.csv", index=False)

df = pd.read_csv("./collated_dataset.csv")

In [24]:
# Reduce admit and discharge times to dates
df = df.sort_values(by=["admittime"]).reset_index(drop=True)
df["admitdate"] = pd.to_datetime(df["admittime"]).dt.date.apply(lambda x: str(x))
df["dischdate"] = pd.to_datetime(df["dischtime"]).dt.date.apply(lambda x: str(x))
df.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,microbiologyevents,pharmacy,poe,prescriptions,procedures_icd,services,transfers,icustays,admitdate,dischdate
0,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaN,EW EMER.,P35SU0,PACU,HOME,Other,...,"[{'microevent_id': 558, 'subject_id': 10001725...","[{'subject_id': 10001725, 'hadm_id': 25563031,...","[{'poe_id': '10001725-48', 'poe_seq': 48, 'sub...","[{'subject_id': 10001725, 'hadm_id': 25563031,...","[{'subject_id': 10001725, 'hadm_id': 25563031,...","[{'subject_id': 10001725, 'hadm_id': 25563031,...","[{'subject_id': 10001725, 'hadm_id': 25563031....","[{'subject_id': 10001725, 'hadm_id': 25563031,...",2110-04-11,2110-04-14
1,10008454,20291550,2110-11-30 06:31:00,2110-12-10 15:53:00,NaN,EW EMER.,P77BSD,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,"[{'microevent_id': 3030, 'subject_id': 1000845...","[{'subject_id': 10008454, 'hadm_id': 20291550,...","[{'poe_id': '10008454-137', 'poe_seq': 137, 's...","[{'subject_id': 10008454, 'hadm_id': 20291550,...","[{'subject_id': 10008454, 'hadm_id': 20291550,...","[{'subject_id': 10008454, 'hadm_id': 20291550,...","[{'subject_id': 10008454, 'hadm_id': 20291550....","[{'subject_id': 10008454, 'hadm_id': 20291550,...",2110-11-30,2110-12-10
2,10004422,21255400,2111-01-15 14:55:00,2111-01-25 15:00:00,NaN,EW EMER.,P1037P,PROCEDURE SITE,HOME HEALTH CARE,Medicare,...,"[{'microevent_id': 1882, 'subject_id': 1000442...","[{'subject_id': 10004422, 'hadm_id': 21255400,...","[{'poe_id': '10004422-100', 'poe_seq': 100, 's...","[{'subject_id': 10004422, 'hadm_id': 21255400,...","[{'subject_id': 10004422, 'hadm_id': 21255400,...","[{'subject_id': 10004422, 'hadm_id': 21255400,...","[{'subject_id': 10004422, 'hadm_id': 21255400....","[{'subject_id': 10004422, 'hadm_id': 21255400,...",2111-01-15,2111-01-25
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,...,"[{'microevent_id': 2474, 'subject_id': 1000605...","[{'subject_id': 10006053, 'hadm_id': 22942076,...","[{'poe_id': '10006053-41', 'poe_seq': 41, 'sub...","[{'subject_id': 10006053, 'hadm_id': 22942076,...","[{'subject_id': 10006053, 'hadm_id': 22942076,...","[{'subject_id': 10006053, 'hadm_id': 22942076,...","[{'subject_id': 10006053, 'hadm_id': 22942076....","[{'subject_id': 10006053, 'hadm_id': 22942076,...",2111-11-13,2111-11-15
4,10035631,29462354,2112-09-17 19:13:00,2112-10-17 01:41:00,NaN,DIRECT EMER.,P45GUA,PHYSICIAN REFERRAL,HOME,Other,...,"[{'microevent_id': 11910, 'subject_id': 100356...","[{'subject_id': 10035631, 'hadm_id': 29462354,...","[{'poe_id': '10035631-269', 'poe_seq': 269, 's...","[{'subject_id': 10035631, 'hadm_id': 29462354,...","[{'subject_id': 10035631, 'hadm_id': 29462354,...","[{'subject_id': 10035631, 'hadm_id': 29462354,...","[{'subject_id': 10035631, 'hadm_id': 29462354....",[],2112-09-17,2112-10-17


In [25]:
# We'd like to look at the vitals of admitted patients to see if there's a pattern that can be observed with their demographics and outcomes.
# These vitals are recorded in the dataset as OMR observations

# Parse the OMR data into df format
df["omr"] = df["omr"].apply(
    lambda row:
    pd.DataFrame(ast.literal_eval(row))
    if type(ast.literal_eval(row)) == list
    else pd.DataFrame([ast.literal_eval(row)])
)
# Clean and pivot it by indicator (height, weight, etc.)
df["omr"] = df["omr"].apply(
    lambda row_df:
    row_df
        .drop_duplicates(subset=["subject_id", "chartdate"]) # keep only one set of observations per person per day
        .sort_values(by=["chartdate"])
        .rename(columns={'result_name': 'index'})
        .pivot(index=["chartdate"], columns='index', values='result_value') # convert to a wide df
        .reset_index()
        .rename(columns={'index': 'chartdate'})
)
df.loc[0, "omr"]

index,chartdate,BMI (kg/m2),Blood Pressure,Height (Inches),Weight (Lbs)
0,2109-10-15,NaN,NaN,NaN,157
1,2110-01-04,NaN,NaN,NaN,152
2,2110-03-02,NaN,NaN,NaN,151
3,2110-04-08,NaN,142/72,NaN,NaN
4,2110-04-19,NaN,117/73,NaN,NaN
...,...,...,...,...,...
61,2114-05-29,27.6,NaN,NaN,NaN
62,2114-06-12,NaN,NaN,NaN,149
63,2114-08-28,NaN,NaN,62,NaN
64,2114-09-21,NaN,NaN,NaN,144


In [26]:
# How many observations of each metric are there for each patient?
df["omr_counts"] = df["omr"].apply(
    lambda row_df:
    [(row_df
      [~row_df[column].isna()] # filter a particular type of observation
      [column].count() # count the number of observations of that type
      if column in row_df.columns else 0)
     for column in df.loc[0, "omr"].columns[1:]])
df["omr_counts"].head()

0    [22, 12, 5, 27]
1       [0, 0, 1, 0]
2       [0, 1, 0, 2]
3       [0, 0, 0, 0]
4     [38, 7, 51, 3]
Name: omr_counts, dtype: object

In [27]:
# How many observations of each metric are there for each patient?
# Unfortunately, since there are no consistent, and rarely any at all, observations during the patients' stays, time series analysis becomes entirely untenable.
df["omr_counts_during_stay"] = df.apply(
    lambda row:
    [row["omr"]
     [(~row["omr"][column].isna()) & (~row["omr"]["chartdate"].isna())] # filter a type of (valid only) observation
     [(row["omr"]["chartdate"] >= row["admitdate"]) & (row["omr"]["chartdate"] <= row["dischdate"])] # filter observations during hospital stay
     [column].count() # count the number of observations of that type
     if column in row["omr"].columns else 0
     for column in df.loc[0, "omr"].columns[1:]], axis=1)
df["omr_counts_during_stay"].head()

0    [0, 0, 0, 0]
1    [0, 0, 0, 0]
2    [0, 0, 0, 0]
3    [0, 0, 0, 0]
4    [0, 0, 0, 0]
Name: omr_counts_during_stay, dtype: object

In [29]:
# Take the sub-datasets (patients, hosp, poe, etc.), flatten them and append them to the main dataframe
def normalize_jsons(column, dropper, sorter):
    data = []
    for datum in column:
        json_df = pd.json_normalize(ast.literal_eval(datum)[0])
        data.append(json_df)
    column_df = pd.concat(data, ignore_index=True).drop_duplicates(subset=dropper).sort_values(by=sorter).reset_index(drop=True)
    return column_df

In [30]:
df = pd.merge(df, normalize_jsons(df["patients"], ["subject_id"], ["anchor_year", "subject_id"]), how='left', on="subject_id", suffixes=("", ""))
df.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,...,icustays,admitdate,dischdate,omr_counts,omr_counts_during_stay,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10001725,25563031,2110-04-11 15:08:00,2110-04-14 15:00:00,NaN,EW EMER.,P35SU0,PACU,HOME,Other,...,"[{'subject_id': 10001725, 'hadm_id': 25563031,...",2110-04-11,2110-04-14,"[22, 12, 5, 27]","[0, 0, 0, 0]",F,46,2110,2011 - 2013,
1,10008454,20291550,2110-11-30 06:31:00,2110-12-10 15:53:00,NaN,EW EMER.,P77BSD,EMERGENCY ROOM,HOME HEALTH CARE,Other,...,"[{'subject_id': 10008454, 'hadm_id': 20291550,...",2110-11-30,2110-12-10,"[0, 0, 1, 0]","[0, 0, 0, 0]",F,26,2110,2011 - 2013,
2,10004422,21255400,2111-01-15 14:55:00,2111-01-25 15:00:00,NaN,EW EMER.,P1037P,PROCEDURE SITE,HOME HEALTH CARE,Medicare,...,"[{'subject_id': 10004422, 'hadm_id': 21255400,...",2111-01-15,2111-01-25,"[0, 1, 0, 2]","[0, 0, 0, 0]",M,78,2111,2011 - 2013,
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,...,"[{'subject_id': 10006053, 'hadm_id': 22942076,...",2111-11-13,2111-11-15,"[0, 0, 0, 0]","[0, 0, 0, 0]",M,52,2111,2014 - 2016,2111-11-15
4,10035631,29462354,2112-09-17 19:13:00,2112-10-17 01:41:00,NaN,DIRECT EMER.,P45GUA,PHYSICIAN REFERRAL,HOME,Other,...,[],2112-09-17,2112-10-17,"[38, 7, 51, 3]","[0, 0, 0, 0]",M,63,2112,2011 - 2013,2116-03-12


In [31]:
def plot_histogram(x1, bins, title, x_label, y_label, color1, lcolor1, x2=None, color2=None, lcolor2=None):
    ax = x1.hist(bins=bins, xlabelsize=10, ylabelsize=6, color=color1)
    plt.axvline(x1.median(), color=lcolor1, linestyle='dashed', linewidth=2)

    if x2 is not None:
        ax2 = x2.hist(bins=bins, xlabelsize=10, ylabelsize=6, color=color2)
        plt.axvline(x2.median(), color=lcolor2, linestyle='dashed', linewidth=2)

    ax.set_title(title, weight='bold')
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.grid(False)
    plt.show()